In [1]:
"""Try to test simple image loaders for compressed and non-compressed data"""

import numpy as np
import fabio
import hdf5plugin
import silx.io, silx.utils
from segmentation import LocalBGCorrector, load_test_image
from silx.opencl.codec.bitshuffle_lz4 import BitshuffleLz4

from matplotlib import pyplot as plt
%matplotlib widget


/home/esrf/james1997a/Code/Anri/.conda/lib/python3.11/site-packages/pytools/persistent_dict.py:52: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


In [2]:
eiger_file = "/data/id11/inhouse2/test_data_3DXRD/S3DXRD/FeAu/RAW_DATA/FeAu_0p5_tR_nscope/FeAu_0p5_tR_nscope_top_200um/FeAu_0p5_tR_nscope_top_200um.h5"
frelon_file = "/data/id11/inhouse2/test_data_3DXRD/TDXRD/FeAu/RAW_DATA/FeAu_0p5_tR/FeAu_0p5_tR_ff1/FeAu_0p5_tR_ff1.h5"
frelon_mask = "/data/id11/inhouse2/test_data_3DXRD/TDXRD/FeAu/pars/mask.edf"
frelon_flat = "/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/F36_Nov2023.edf"
frelon_dark = "/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/dark_20240416.edf"

In [3]:
mask = fabio.open(frelon_mask).data.astype(bool)
bg = None
flat = fabio.open(frelon_flat).data.astype(np.float32)
dark = fabio.open(frelon_dark).data.astype(np.float32)

corrfunc = LocalBGCorrector(
    threshold=100,
    smoothsigma=1.0,
    bgc=0.9, 
    bg=bg,
    dark=dark,
    flat=flat,
    mask=mask,
    m_offset_thresh=70,
    m_ratio_thresh=150,
)

In [4]:
def segment_frames(frames_iterator, corrector_function, startidx, stopidx):
    for idx in range(startidx, stopidx+1):
        image = frames_iterator[idx]
        corrected_image = corrector_function(image.astype(np.float32))
        yield corrected_image

In [5]:
def read_and_correct():
    data_path = '1.1/measurement/frelon3'

    with silx.io.h5py_utils.File(frelon_file, 'r') as f:
        dset = f[data_path]

        for corrected_image in segment_frames(dset, corrfunc, 0, 100):
            pass

In [6]:
%prun -s cumulative read_and_correct()

         19485 function calls (19484 primitive calls) in 20.473 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   20.473   20.473 {built-in method builtins.exec}
        1    0.001    0.001   20.473   20.473 <string>:1(<module>)
        1    0.038    0.038   20.472   20.472 3858188479.py:1(read_and_correct)
      102    0.037    0.000   20.433    0.200 3877884579.py:1(segment_frames)
      101    6.501    0.064   19.961    0.198 segmentation.py:186(__call__)
      101    0.002    0.000    6.669    0.066 _filters.py:326(gaussian_filter)
      202    0.002    0.000    6.620    0.033 _filters.py:257(gaussian_filter1d)
      202    0.002    0.000    6.610    0.033 _filters.py:125(correlate1d)
      202    6.606    0.033    6.606    0.033 {built-in method scipy.ndimage._nd_image.correlate1d}
      101    0.003    0.000    6.547    0.065 _filters.py:1119(uniform_filter)
      202    0.002    0.000   